---
title: Concatenating HS Founder Epigenomes
author: Sabrina Mi
date: 12/6/2023
---

## Initialize Gene-Interval Mapping

In [25]:
import pandas as pd
import h5py
import os

In [40]:

with open("/home/s1mi/Github/deep-learning-in-genomics/posts/2023-11-07-HS-founder-epigenomes/Br_expressed_genes.txt", "r") as f:
    gene_list = f.read().splitlines()
with open("/home/s1mi/Github/deep-learning-in-genomics/posts/2023-11-07-HS-founder-epigenomes/metadata/intervals.txt", "r") as f:
    intervals = f.read().splitlines()
mapping = pd.DataFrame({"gene": gene_list, "interval": intervals})
mapping['chromosome'] = mapping['interval'].str.split('_').str[0].str[3:]


## Concatenate by Chromosome

In [60]:
predictions_dir = "/eagle/AIHPC4Edu/sabrina/Br_predictions/predictions_folder/HS_founders_genome_wide/predictions_2023-11-27/enformer_predictions"
output_dir = "/eagle/AIHPC4Edu/sabrina/Br_predictions/HS_founder_epigenomes"
gene_dict = mapping.groupby('chromosome')['gene'].apply(list).to_dict()
founders = os.listdir(predictions_dir)

In [100]:
for N in gene_dict.keys():
    output_path = os.path.join(output_dir, "human", f"chr{N}_genes.h5")
    with h5py.File(output_path, "w"):
        pass
    output_path = os.path.join(output_dir, "mouse", f"chr{N}_genes.h5")
    with h5py.File(output_path, "w"):
        pass


In [111]:
def concatenate_predictions(N, completed):
    for gene in gene_dict[N]:
        if gene in completed:
            continue
        interval = mapping[mapping["gene"] == gene]['interval'].item()
        for individual in founders:
            input_path = os.path.join(predictions_dir, individual, "haplotype0",f'{interval}_predictions.h5')
            with h5py.File(input_path, "r") as input:
                for model in input.keys():
                    output_path = os.path.join(output_dir, model, f"chr{N}_genes.h5")
                    with h5py.File(output_path, "a") as output:
                        if gene not in output:
                            gene_group = output.create_group(gene)
                        else:
                            gene_group = output[gene]
                        if individual not in gene_group:
                            gene_group.create_dataset(individual, data=input[model][:])
        completed.append(gene)


            

In [110]:
output_path = os.path.join(output_dir, "human", f"chr20_genes.h5")
with h5py.File(output_path, "r") as hf:
    for key, value in hf.items():
        print(key, value)
output_path = os.path.join(output_dir, "mouse", f"chr20_genes.h5")
with h5py.File(output_path, "r") as hf:
    for key, value in hf.items():
        print(key, value)

ENSRNOG00000026051 <HDF5 group "/ENSRNOG00000026051" (2 members)>
ENSRNOG00000031607 <HDF5 group "/ENSRNOG00000031607" (8 members)>
ENSRNOG00000026051 <HDF5 group "/ENSRNOG00000026051" (2 members)>
ENSRNOG00000031607 <HDF5 group "/ENSRNOG00000031607" (8 members)>


In [112]:
completed_genes = []
concatenate_predictions("20", completed_genes)